In [112]:
import numpy as np
import scipy.special as sp
from numpy import linalg as al
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import csv

from itertools import combinations
from random import random , choice

import winsound

#conjuntos de nodos

all_nodes = ['C3', 'C4', 'CZ', 'EMG', 'F3', 'F4', 'F7', 'F8', 'FP1', 'FP2', 'FZ', 'LOG', 'O1', 'O2', 'P3', 'P4', 'PZ', 
             'ROG', 'T3', 'T4', 'T5', 'T6']

frontal=['F3', 'F4', 'F7', 'F8', 'FP1', 'FP2', 'FZ','LOG','ROG']
central=['C3', 'C4', 'CZ']
izquierdo = ['C3', 'F3', 'F7', 'FP1', 'LOG', 'O1','P3','T3',  'T5']
medio = ['CZ','FZ','PZ']
derecho = ['C4','F4', 'F8', 'FP2',  'O2', 'P4', 'ROG', 'T4', 'T6']

zonas_=['all_nodes','izquierdo','derecho','medio','izquierdo+medio','derecho+medio','izquierdo+derecho']

#-------------------------------------------------------

#funciones varias

def graph_from_csv(filename):
    df = pd.read_csv(filename, index_col=0)
    return nx.from_pandas_adjacency(df)

def positiv(graph):
    for i in graph.nodes:
        for j in graph.nodes:
            if i !=j:
                graph[i][j]['weight'] = abs(graph[i][j]['weight'])
            else:
                graph[i][j]['weight'] = 0
    return(graph)

def random_weighted_graph(the_nodes):
    G = nx.Graph()
    G.add_nodes_from(the_nodes)
    all_edges = combinations(the_nodes, 2)
    elist = [(e[0], e[1], random()) for e in all_edges]
    G.add_weighted_edges_from(elist)
    return G

def selected_graph(graph, param, bigger_than=True):
    vs = list(graph.nodes())
    n = graph.order()
    out = nx.Graph()
    out.add_nodes_from(vs)
    if bigger_than:
        out.add_edges_from([(vs[i], vs[j]) for i in range(n) for j in range(n) if i != j
                            and graph[vs[i]][vs[j]]['weight'] > param])
    else:
        out.add_edges_from([(vs[i], vs[j]) for i in range(n) for j in range(n) if i != j
                            and graph[vs[i]][vs[j]]['weight'] < param])
    return out

def eigenvalores(A):
        av=list(al.eigh(A))
    
        av[0]=av[0].astype(float)
        av[1]=av[1].astype(float)
    
        for i in range(len(av[0]-1)):
            av[0][i]=round(float(av[0][i]),5)
    
        return av[0]
    
def eigenvectores(A):
        av=list(al.eigh(A))
    
        av[1]=av[1].astype(float)
    
        for i in range(len(av[1]-1)):
            for j in range(len(av[1][i]-1)):
                av[1][i][j]=round(float(av[1][i][j]),3)
    
        return av[1].T
    
def Fiedler(G):
        lap=nx.laplacian_matrix(G)
        print("valor Fiedler")
        print(eigenvalores(lap.A)[1])
        print("vector Fiedler")
        print(eigenvectores(lap.A)[1])
        print("los eigenvalores")
        print(eigenvalores(lap.A))
    
    # Función genérica para el cociente de Rayleigh para una matriz y un vector,
    
def rayleigh(A,v):
        numerador = np.dot(np.dot(A,v),v)
        denominador = np.dot(v,v)
        S = numerador/denominador
        return S
    
def peso_grafica(G):
    long=len(G.nodes)
    peso=sum([list(G.degree(weight='weight'))[k][1]/2 for k in range(long)])
    return(peso)
    
    
def vector_candidato(G): #un vector sencillo candidato a tener cociente de Rayleigh pequeño
    grados=[list(G.degree(weight='weight'))[k][1] for k in range(len(G))]

    nodo_uno=list(G.nodes)[np.argmin(np.array(grados))]

    grados1=[list(G.subgraph(set(G.nodes)-{nodo_uno}).degree(weight='weight'))[k][1] for k in range(len(G)-1)]
    #e2=np.where(np.array(grados1)==min(grados1))
    nodo_dos=list(G.nodes)[np.argmin(np.array(grados1))]

    v=[0]*len(G)
    v[np.argmin(np.array(grados))]=1
    v[np.argmin(np.array(grados1))]=-1
    return(v)

def un_maximo_local(G):
    maximal_ac = conectividad_algebraica(G)
    print(maximal_ac)
    
    subgrafica=G
    for i in range(1,len(G)):
        size=len(subgrafica)-1
        print("\r",size," ",end="")
        if size<maximal_ac:
            break
        subgrafica = G.subgraph(quitar_uno_express(subgrafica))
        ac = conectividad_algebraica(subgrafica)
        if ac>maximal_ac: 
            maximal_ac = ac
            grafica_optima = subgrafica
            print(maximal_ac)
    subgrafica = grafica_optima
    nodos_nuevos=set(grafica_optima.nodes)
    #print("\r","    \n","\rde bajada candidato a máximo en",nodos_nuevos)
    #print("con valor Fiedler igual a",maximal_ac)
    #print("número de nodos", len(grafica_optima))
    for i in range(len(grafica_optima),len(G)-1):
        subgrafica=poner_uno_express(G,subgrafica)
        if conectividad_algebraica(subgrafica)>maximal_ac:
            maximal_ac = conectividad_algebraica(subgrafica)
            grafica_optima = subgrafica
            print(i,maximal_ac)
            
    nodos_nuevos=set(grafica_optima.nodes)
    print("\r","    \n","\rcandidato a máximo en",nodos_nuevos)
    print("con valor Fiedler igual a",maximal_ac)
    print("número de nodos", len(grafica_optima))
    return ([grafica_optima,maximal_ac])

def otro_maximo_local(G,pasos=11,m=11):
    #m = int(np.ceil(len(G.nodes)/2))
    grados = np.array([list(G.degree(weight='weight'))[k][1] for k in range(len(G.nodes))])
    nuevos_nodos = list(np.array(G.nodes)[np.argsort(grados)[-m:]])
    optima = G.subgraph(nuevos_nodos)
    subgraf = optima
    maximal_ac = conectividad_algebraica(subgraf)
    for i in range(pasos):
        if len(subgraf.nodes)==len(G.nodes) or len(subgraf.nodes)==1:
            break
        tres_graficas=[quitar_uno_express(subgraf),optima,poner_uno_express(G,subgraf)]
        dos_graficas=[quitar_uno_express(subgraf),poner_uno_express(G,subgraf)]
        tres_conectividades=np.array([conectividad_algebraica(k) for k in tres_graficas])
        dos_conectividades=np.array([conectividad_algebraica(k) for k in dos_graficas])
        optima=tres_graficas[np.argmax(tres_conectividades)]
        subgraf=dos_graficas[np.argmax(dos_conectividades)]
        
    return(optima)
    
        
    
def pares_vetados(G,cota): #pares de nodos que no pueden estar en subgráfica con valor Fiedler mayor a cota
    lap = nx.laplacian_matrix(G)
    pares = [tuple([aristas[0],aristas[1]]) for aristas in G.edges if aristas[0]!=aristas[1]]
    vetados = []
    for pareja in pares:
        v = [0]*len(G.nodes)
        for k in range(len(G.nodes)):
            if list(G.nodes)[k]==pareja[0]:
                v[k]=-1
            if list(G.nodes)[k]==pareja[1]:
                v[k]=1
        cociente = rayleigh(lap.A,v)
        if cociente<cota:
            vetados.append(pareja)
    return(vetados)
    
''' 
def subgrafica_optima_ac(G):#Dada una gráfica con pesos, regresa la subgráfica con máximo valor Fiedler
    

    return(G.subgraph(nodos_selectos))    
'''

#--------------------------------------------------------    

#funciones particulares para explorar valores Fiedler de subgráficas de los datos del PSG

def quitar_uno(G):
    print("El valor Fiedler de toda la gráfica es",round(conectividad_algebraica(positiv(G)),5))
    print("\n")

    valores_nodos=dict(zip(G.nodes,eigenvectores(lap.A)[1]))
    print("El eigenvector Fiedler correspondiente es",valores_nodos)

    todos=set(G.nodes)
    nuevas=list([round(conectividad_algebraica(positiv(G).subgraph(todos-{nodo})),5) for nodo in list(todos)])
    print("\n")
    print("Valores Fiedler, quitando el nodo que se indica: ")
    
    d=dict(zip(list(todos),nuevas))
    print(d)
    print("\n")
    
    i = 0
    for nodo in list(todos):
        #print(nodo)
        if nuevas[i] == max(nuevas):
            print(list(todos)[i],"You are the weakest link")
            print("Quitando",list(todos)[i],"el valor Fiedler alcanza su máximo, que es",nuevas[i])
            break
        i = i+1
    return()
    #nx.draw_circular(positiv(G).subgraph(todos-{nodo}),with_labels=True)  
    
def quitar_uno_express(S): #regresa la subgrafica con un nodo menos y mayor Fiedler
    todos = tuple(S.nodes)
    reducidos = [conectividad_algebraica(positiv(S).subgraph(set(todos)-{nodo})) for nodo in todos]
    weak_link={todos[np.argmax(reducidos)]}
    nuevos_nodos = tuple(set(todos)-weak_link)  
    
    return(S.subgraph(nuevos_nodos))

def poner_uno_express(G,S): #regresa la subgrafica con mayor Fiedler de entre las gráficas S más otro nodo
    sobrantes=tuple(set(G.nodes)-set(S.nodes))
    graficas_aumentadas = [G.subgraph(tuple(set.union(set(S.nodes),{nodo_extra}))) for nodo_extra in sobrantes]
    vf_aumentadas = [conectividad_algebraica(aumentadas) for aumentadas in graficas_aumentadas]
    optimo = np.argmax(vf_aumentadas)
    
    return(graficas_aumentadas[optimo])    
    
def regiones_especificas(G):
    subs=list()

    [subs.append(positiv(G).subgraph(zona)) for zona in zonas] 

    print("\n\n Valores Fiedler por regiones\n")

    for i in range(len(zonas)):
        print(zonas_[i],"         ",round(conectividad_algebraica(subs[i]),5))

def conectividad_algebraica(G):
    laplace=nx.laplacian_matrix(G)
    return(eigenvalores(laplace.A)[1])

In [127]:
G = positiv(graph_from_csv('./variaciones/CorrelacionesREM/correls_FGH.csv'))
print("valor Fiedler de toda la gráfica es", conectividad_algebraica(G))

ng=otro_maximo_local(G)
print("subgrafica con nodos", list(ng.nodes))
print("longitud",len(ng.nodes))
print("valor Fiedler", conectividad_algebraica(ng))
cota=conectividad_algebraica(otro_maximo_local(G))

len(pares_vetados(G,conectividad_algebraica(ng)))


valor Fiedler de toda la gráfica es 0.26219
subgrafica con nodos ['C3', 'C4', 'CZ', 'EMG', 'O1', 'O2', 'PZ', 'T3', 'T4', 'T5', 'T6']
longitud 11
valor Fiedler 10.01014


176

In [120]:
G = positiv(graph_from_csv('./variaciones/CorrelacionesREM/correls_GURM.csv'))
otro_maximo_local(G)


[('C3', 'C3'),
 ('C3', 'C4'),
 ('C3', 'EMG'),
 ('C3', 'F3'),
 ('C3', 'F4'),
 ('C3', 'F7'),
 ('C3', 'F8'),
 ('C3', 'FP2'),
 ('C3', 'FZ'),
 ('C3', 'LOG'),
 ('C3', 'O1'),
 ('C3', 'O2'),
 ('C3', 'P3'),
 ('C3', 'P4'),
 ('C3', 'PZ'),
 ('C3', 'ROG'),
 ('C3', 'T3'),
 ('C3', 'T4'),
 ('C3', 'T5'),
 ('C3', 'T6'),
 ('C4', 'C4'),
 ('C4', 'EMG'),
 ('C4', 'F3'),
 ('C4', 'F4'),
 ('C4', 'F7'),
 ('C4', 'F8'),
 ('C4', 'FP2'),
 ('C4', 'FZ'),
 ('C4', 'LOG'),
 ('C4', 'O1'),
 ('C4', 'O2'),
 ('C4', 'P3'),
 ('C4', 'P4'),
 ('C4', 'PZ'),
 ('C4', 'ROG'),
 ('C4', 'T3'),
 ('C4', 'T4'),
 ('C4', 'T5'),
 ('C4', 'T6'),
 ('EMG', 'EMG'),
 ('EMG', 'F3'),
 ('EMG', 'F4'),
 ('EMG', 'F7'),
 ('EMG', 'F8'),
 ('EMG', 'FP2'),
 ('EMG', 'FZ'),
 ('EMG', 'LOG'),
 ('EMG', 'O1'),
 ('EMG', 'O2'),
 ('EMG', 'P3'),
 ('EMG', 'P4'),
 ('EMG', 'PZ'),
 ('EMG', 'ROG'),
 ('EMG', 'T3'),
 ('EMG', 'T4'),
 ('EMG', 'T5'),
 ('EMG', 'T6'),
 ('F3', 'F3'),
 ('F3', 'F4'),
 ('F3', 'F7'),
 ('F3', 'F8'),
 ('F3', 'FP2'),
 ('F3', 'FZ'),
 ('F3', 'LOG'),
 ('F3', 

In [42]:
G = positiv(graph_from_csv('./variaciones/CorrelacionesREM/correls_JANA.csv'))
grados=np.array([list(G.degree(weight='weight'))[k][1] for k in range(len(G.nodes))])
indices=np.argsort(grados)[-4:]
print(tuple(G.nodes))    
print(grados)
print(indices)
print(list(np.array(G.nodes)[indices]))

('C3', 'C4', 'CZ', 'EMG', 'F3', 'F4', 'F7', 'F8', 'FP1', 'FP2', 'FZ', 'LOG', 'O1', 'O2', 'P3', 'P4', 'PZ', 'ROG', 'T3', 'T4', 'T5', 'T6')
[15.25052702 14.21114371 15.72130577 12.35812588 13.74643552 16.00065092
  9.62850481 15.61639972 10.36270052 10.16914444 13.91347675  6.86416534
 15.47393538 15.48469712 15.73789102 15.23286567 15.56907811  8.47232468
 14.2441074  15.58225088 16.35941026 15.23524   ]
[ 2 14  5 20]
['CZ', 'P3', 'F4', 'T5']
